In [1]:
import pandas as pd
import numpy as np
import h5py
import datetime
import matplotlib.pyplot as plt
from scipy import interpolate
file_name = "simulation_test2.h5"
f = h5py.File(file_name, "r")

In [2]:
def create_list_of_datasets_in_h5file(h5_file):
    """
    Returns a list of the paths of all datasets in an h5 file.
    Can also be used to list all datasets under a given node (e.g., f['session001']).  In this case,
    the 'session001' is not included in the output paths.
    This comes from:
    https://stackoverflow.com/questions/44883175/how-to-list-all-datasets-in-h5py-file
    :param f:
    :return:
    """
    keys = []
    h5_file.visit(lambda key: keys.append(key) if type(h5_file[key]) is h5py._hl.dataset.Dataset else None)
    return keys


def create_list_of_groups_in_node(node):
    keys = []
    node.visit(lambda key: keys.append(key) if type(node[key]) is h5py._hl.group.Group else None)
    return keys

def create_list_of_items_in_node(item_type, h5_file, node):
    keys = []
    if item_type == "group":
        my_type = h5py._hl.group.Group
    if item_type == "dataset":
        my_type = h5py._hl.dataset.Dataset
    h5_file[node].visit(lambda key: keys.append(key) if type(h5_file[node][key]) is my_type else None)
    return keys

def create_list_of_insertions_in_node(h5_file, node):
    insertions = []
    all_groups = create_list_of_items_in_node("group", h5_file, node)
    for group in all_groups:
        if group[-6:-3] == 'ins':
            insertions.append(group)
    return insertions

def create_session_insertion_dict(h5_file):
    keys = list(h5_file.keys())
    key_dict = {}
    for key in keys:
        # session_number = int(key[-3:])
        key_dict[key] = create_list_of_insertions_in_node(h5_file, key)
    print(key_dict)
    return key_dict

def create_list_of_calibrations_in_node(h5_file, node):
    calibrations = []
    all_groups = create_list_of_items_in_node("group", h5_file, node)
    for group in all_groups:
        if group[-6:-3] == 'cal':
            calibrations.append(group)
    return calibrations


def create_session_calibration_dict(h5_file):
    keys = list(h5_file.keys())
    key_dict = {}
    for key in keys:
        key_dict[key] = create_list_of_calibrations_in_node(h5_file, key)
    print(key_dict)
    return key_dict

def get_wavelengths_vector(h5_file, session, insertion, spectrometer):
    if spectrometer == "spectrometer1":
        vector_attr = "spec1_wavelengths_vector"
    elif spectrometer == "spectrometer2":
        vector_attr = "spec2_wavelengths_vector"
    else:
        print("spectrometer not specified")
        vector_attr = ""

    insertion_group = h5_file[f"{session}/{insertion}"]
    wavelengths_vector = insertion_group.parent.attrs[vector_attr][:]
    return wavelengths_vector

def get_absorbance_spectra_in_insertion(h5_file, session, insertion):
    absorbance_string = "spectrometer1/derived/absorbances"
    absorbance_spectra = h5_file[f"{session}/{insertion}/{absorbance_string}"][:]
    print(absorbance_spectra)
    return absorbance_spectra

def get_absorbance_spectra_and_depth_in_insertion(h5_file, session, insertion):
    dset_string = "derived/absorbance_depth"
    dset = h5_file[f"{session}/{insertion}/{dset_string}"]
    absorbance_spectra = dset[:, :-2]
    depths = dset[:, -1]
    forces = dset[:, -2]
    print(absorbance_spectra)
    return absorbance_spectra, depths, forces

def get_current_time():
    return np.datetime64(datetime.datetime.now()).astype(np.int64)


def timestamp_list_to_pandas_timestamps(timestamp_list):
    series = pd.Series(timestamp_list).astype('<M8[us]')
    return series

In [98]:
def calculate_depth_bins(depth_min=0.0, depth_max=100.0, depth_increment=5.0):
    """
    Calculates arrays that represent the intervals for regular depth increments (in cm).
    :param depth_min: The minimum value for the first depth increment
    :param depth_max: The maximum value for the last depth increment
    :param depth_increment: The depth increment
    :return: an np.array representing the bin minima, and an np.array representing the bin centers.
    """
    bin_mins = np.arange(depth_min, depth_max, depth_increment)
    bin_centers = bin_mins + depth_increment/2.0
    return bin_mins, bin_centers


def calculate_weighted_averages_by_depth_increment(data, depths, depth_min=0.0, depth_max=100.0, depth_increment=5.0):
    """
    Calculates the weighted average value for a data array relative to a regular depth increment.

    :param data: np.array representing the data to be weighted
    :param depths: np.array representing depths corresponding to the data array
    :param depth_min: The minimum depth for the resulting depth increments (cm)
    :param depth_max: the maximum depth for the resulting depth increments (cm)
    :param depth_increment: The size of the depth increment bins (cm)
    :return: np.array representing the weighted averages, np.array of the bin depth minima,
    np.array of the bin depth centers.
    """
    bin_mins, bin_centers = calculate_depth_bins(depth_min, depth_max, depth_increment)
    weighted_data = []
    for j in range(bin_mins.shape[0]):
        bin_number = j
        weights = []
        bin_data = []
        bin_min = bin_mins[bin_number]
        # sets data for depth increment to zero when bin is outside data range
        if bin_min > depths.max():
            bin_mins = bin_mins[:j]
            bin_centers = bin_centers[:j]
            break
        else:
            for i in range(data.shape[0]):
                if depths[i] < bin_min:
                    pass
                if depths[i] >= bin_min:
                    if i == 0:
                        weight = depths[i] - bin_min
                    elif depths[i-1] <= bin_min:
                        weight = depths[i] - bin_min
                    else:
                        weight = depths[i] - depths[i-1]
                    weights.append(weight)
                    # gets rid of the portion of the data that is outside the depth increment
                    if sum(weights) >= depth_increment:
                        weights.pop()
                        weights.append(depth_increment - sum(weights))
                        bin_data.append(data[i])
                        break
                    elif sum(weights) < depth_increment:
                        bin_data.append(data[i])
        bin_weighted_average = np.average(bin_data, weights=weights, axis=0)
        weighted_data.append(bin_weighted_average)
    return np.array(weighted_data), bin_mins, bin_centers


In [4]:
dset_string = 'session012/cal001/ins003/derived/absorbance_depth'

In [5]:
dset = f[dset_string]

In [6]:
dset[:]

array([[ 0.84194035,  0.8290654 ,  0.83331382, ...,  1.35837461,
        29.03639952,  0.20817084],
       [ 0.84240488,  0.83504382,  0.83149244, ...,  1.37684916,
        50.03308722,  2.63598446],
       [ 0.84373123,  0.8329363 ,  0.83171938, ...,  1.3348489 ,
        28.24391123,  4.38395732],
       ...,
       [ 0.84354403,  0.83239759,  0.83515361, ...,  1.35235225,
        41.33606119, 92.54212338],
       [ 0.84084816,  0.83142671,  0.8378247 , ...,  1.32789153,
        50.96959953, 94.45680687],
       [ 0.84444666,  0.82868655,  0.83209898, ...,  1.38365439,
        49.95236415, 96.00391662]])

In [7]:
absorbances = dset[:, :-2]
forces = dset[:, -2]
depths = dset[:, -1]

In [8]:
depths

array([ 0.20817084,  2.63598446,  4.38395732,  5.64479418,  7.24765674,
        9.24296437, 11.15246335, 13.68255466, 15.69645544, 17.6474673 ,
       19.08876747, 20.90147065, 22.64758332, 24.6440933 , 26.27080448,
       28.2176177 , 30.08048713, 32.09185115, 34.20952293, 36.22524898,
       38.61003438, 41.15772535, 42.83106477, 45.29170716, 47.21422079,
       49.42790932, 51.51423622, 53.03521615, 55.22806467, 57.41845705,
       58.72604148, 60.72810734, 63.0645109 , 65.36665788, 67.48522759,
       68.71543831, 70.19205888, 71.96052662, 73.65067446, 75.42026676,
       77.36244217, 79.2188693 , 80.809298  , 83.22955187, 84.734879  ,
       86.49653278, 88.43715241, 90.32605912, 92.54212338, 94.45680687,
       96.00391662])

In [9]:
forces

array([29.03639952, 50.03308722, 28.24391123, 66.92099891, 35.35176692,
       67.96905686, 47.16921006, 60.97937832, 55.18490803, 50.68878626,
       36.81851123, 50.88855765, 39.14291858, 78.52665892, 44.25279792,
       34.54808736, 49.73265713, 41.80005453, 52.92111244, 55.70904996,
       69.52077824, 57.89832857, 41.73661567, 56.60477145, 48.33527745,
       64.08815095, 55.08365969, 67.30239137, 53.7546155 , 59.71935205,
       54.95027841, 74.05428648, 62.4104803 , 60.4797313 , 37.36613492,
       53.26402307, 48.27332609, 57.78216544, 53.43822676, 52.22873994,
       64.33763003, 62.53099505, 57.76200295, 64.14807149, 34.55366819,
       56.64581284, 33.55902992, 49.19886615, 41.33606119, 50.96959953,
       49.95236415])

In [10]:
absorbances

array([[0.84194035, 0.8290654 , 0.83331382, ..., 1.21914795, 1.16356428,
        1.35837461],
       [0.84240488, 0.83504382, 0.83149244, ..., 1.2003397 , 1.20634406,
        1.37684916],
       [0.84373123, 0.8329363 , 0.83171938, ..., 1.21384865, 1.21594736,
        1.3348489 ],
       ...,
       [0.84354403, 0.83239759, 0.83515361, ..., 1.21448484, 1.21320101,
        1.35235225],
       [0.84084816, 0.83142671, 0.8378247 , ..., 1.23572411, 1.19398532,
        1.32789153],
       [0.84444666, 0.82868655, 0.83209898, ..., 1.19938113, 1.16906148,
        1.38365439]])

In [13]:
weighted_absorbances, bin_mins, bin_centers = calculate_weighted_averages_by_depth_increment(absorbances, depths, 0.0, 100.0, 5.0)

In [14]:
weighted_forces, bin_mins, bin_centers = calculate_weighted_averages_by_depth_increment(forces, depths, 0.0, 100.0, 5.0)

In [15]:
weighted_absorbances.shape

AttributeError: 'list' object has no attribute 'shape'

In [16]:
wabs = np.array(weighted_absorbances)

In [17]:
wabs

array([[0.84320213, 0.83326291, 0.83190135, ..., 1.20952137, 1.20923595,
        1.35588618],
       [0.84019222, 0.83270569, 0.8324477 , ..., 1.23433916, 1.18710789,
        1.34098836],
       [0.8418255 , 0.82982735, 0.83462213, ..., 1.24448333, 1.20003333,
        1.34383033],
       ...,
       [0.84382592, 0.82807597, 0.83459377, ..., 1.22763743, 1.18753708,
        1.37210411],
       [0.84264218, 0.83138039, 0.83562795, ..., 1.2222952 , 1.19977144,
        1.3457    ],
       [0.84444666, 0.82868655, 0.83209898, ..., 1.19938113, 1.16906148,
        1.38365439]])

In [18]:
wabs.shape

(20, 161)

In [19]:
wfor = np.array(weighted_forces)

In [20]:
wfor.shape

(20,)

In [21]:
df_try = pd.DataFrame(data=wabs)

In [22]:
df_try

,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
0,0.843202,0.833263,0.831901,0.833744,0.840334,0.825135,0.825624,0.822003,0.804946,0.818909,...,1.220336,1.270153,1.290030,1.335309,1.204376,1.270656,1.359324,1.209521,1.209236,1.355886
1,0.840192,0.832706,0.832448,0.835993,0.836576,0.824540,0.825283,0.823061,0.802993,0.814984,...,1.234226,1.272436,1.289186,1.326709,1.220906,1.259035,1.366593,1.234339,1.187108,1.340988
2,0.841825,0.829827,0.834622,0.835858,0.836838,0.823496,0.825259,0.823211,0.804676,0.816166,...,1.242538,1.268364,1.308526,1.310733,1.224171,1.261525,1.332410,1.244483,1.200033,1.343830
3,0.844962,0.831654,0.836121,0.836109,0.839911,0.824621,0.825179,0.822863,0.802596,0.819260,...,1.232261,1.276267,1.332737,1.314673,1.233433,1.254714,1.359270,1.239827,1.190060,1.332035
4,0.844953,0.830425,0.838259,0.835237,0.834865,0.824078,0.820861,0.821793,0.804235,0.818778,...,1.217382,1.270360,1.337605,1.302359,1.238767,1.264385,1.364298,1.219246,1.170694,1.354511
5,0.842986,0.829954,0.834005,0.835456,0.837331,0.824301,0.823125,0.823025,0.803151,0.817284,...,1.233520,1.266543,1.331477,1.313922,1.222155,1.264246,1.344110,1.230692,1.187308,1.355166
6,0.844845,0.830811,0.834492,0.833403,0.838310,0.823896,0.823078,0.822826,0.803487,0.814790,...,1.218439,1.276900,1.317835,1.331871,1.211866,1.266534,1.341521,1.215127,1.176776,1.353014
7,0.842162,0.828013,0.833555,0.833949,0.836885,0.823384,0.823271,0.822030,0.803199,0.819456,...,1.220850,1.273558,1.316208,1.329310,1.216431,1.240871,1.351588,1.219945,1.197474,1.358040
8,0.839942,0.826611,0.832969,0.835610,0.836854,0.823476,0.823502,0.821772,0.801042,0.821742,...,1.226188,1.274931,1.319316,1.316141,1.232594,1.245473,1.348477,1.220535,1.179241,1.351113
9,0.842200,0.831449,0.835828,0.834551,0.837295,0.826329,0.823469,0.820606,0.802696,0.819326,...,1.224676,1.271220,1.343420,1.316473,1.211801,1.257726,1.360664,1.209661,1.194785,1.391014


In [23]:
df_try["force"] = wfor

In [24]:
df_try

,0,1,2,3,4,5,6,7,8,9,...,152,153,154,155,156,157,158,159,160,force
0,0.843202,0.833263,0.831901,0.833744,0.840334,0.825135,0.825624,0.822003,0.804946,0.818909,...,1.270153,1.290030,1.335309,1.204376,1.270656,1.359324,1.209521,1.209236,1.355886,43.622265
1,0.840192,0.832706,0.832448,0.835993,0.836576,0.824540,0.825283,0.823061,0.802993,0.814984,...,1.272436,1.289186,1.326709,1.220906,1.259035,1.366593,1.234339,1.187108,1.340988,54.228449
2,0.841825,0.829827,0.834622,0.835858,0.836838,0.823496,0.825259,0.823211,0.804676,0.816166,...,1.268364,1.308526,1.310733,1.224171,1.261525,1.332410,1.244483,1.200033,1.343830,56.269456
3,0.844962,0.831654,0.836121,0.836109,0.839911,0.824621,0.825179,0.822863,0.802596,0.819260,...,1.276267,1.332737,1.314673,1.233433,1.254714,1.359270,1.239827,1.190060,1.332035,47.353218
4,0.844953,0.830425,0.838259,0.835237,0.834865,0.824078,0.820861,0.821793,0.804235,0.818778,...,1.270360,1.337605,1.302359,1.238767,1.264385,1.364298,1.219246,1.170694,1.354511,57.350323
5,0.842986,0.829954,0.834005,0.835456,0.837331,0.824301,0.823125,0.823025,0.803151,0.817284,...,1.266543,1.331477,1.313922,1.222155,1.264246,1.344110,1.230692,1.187308,1.355166,42.427587
6,0.844845,0.830811,0.834492,0.833403,0.838310,0.823896,0.823078,0.822826,0.803487,0.814790,...,1.276900,1.317835,1.331871,1.211866,1.266534,1.341521,1.215127,1.176776,1.353014,48.836847
7,0.842162,0.828013,0.833555,0.833949,0.836885,0.823384,0.823271,0.822030,0.803199,0.819456,...,1.273558,1.316208,1.329310,1.216431,1.240871,1.351588,1.219945,1.197474,1.358040,62.905256
8,0.839942,0.826611,0.832969,0.835610,0.836854,0.823476,0.823502,0.821772,0.801042,0.821742,...,1.274931,1.319316,1.316141,1.232594,1.245473,1.348477,1.220535,1.179241,1.351113,51.928394
9,0.842200,0.831449,0.835828,0.834551,0.837295,0.826329,0.823469,0.820606,0.802696,0.819326,...,1.271220,1.343420,1.316473,1.211801,1.257726,1.360664,1.209661,1.194785,1.391014,56.564260


In [27]:
df_try.iloc[:, :]

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [29]:
type(df_try)

pandas.core.frame.DataFrame

In [30]:
df_try.dtypes

0        float64
1        float64
2        float64
3        float64
4        float64
          ...   
157      float64
158      float64
159      float64
160      float64
force    float64
Length: 162, dtype: object

In [33]:
df_try.to_numpy().dtype

dtype('float64')

In [34]:
df_try.to_numpy()

array([[ 0.84320213,  0.83326291,  0.83190135, ...,  1.20923595,
         1.35588618, 43.6222648 ],
       [ 0.84019222,  0.83270569,  0.8324477 , ...,  1.18710789,
         1.34098836, 54.22844895],
       [ 0.8418255 ,  0.82982735,  0.83462213, ...,  1.20003333,
         1.34383033, 56.2694562 ],
       ...,
       [ 0.84382592,  0.82807597,  0.83459377, ...,  1.18753708,
         1.37210411, 45.35759147],
       [ 0.84264218,  0.83138039,  0.83562795, ...,  1.19977144,
         1.3457    , 46.47390772],
       [ 0.84444666,  0.82868655,  0.83209898, ...,  1.16906148,
         1.38365439, 49.95236415]])

In [40]:
df_try.to_numpy().dtype

dtype('float64')

In [37]:
ftest = h5py.File("test_file.h5", "w")

In [41]:
ftest['test2'] = df_try.to_numpy()

In [42]:
ftest['test2'][:]

array([[ 0.84320213,  0.83326291,  0.83190135, ...,  1.20923595,
         1.35588618, 43.6222648 ],
       [ 0.84019222,  0.83270569,  0.8324477 , ...,  1.18710789,
         1.34098836, 54.22844895],
       [ 0.8418255 ,  0.82982735,  0.83462213, ...,  1.20003333,
         1.34383033, 56.2694562 ],
       ...,
       [ 0.84382592,  0.82807597,  0.83459377, ...,  1.18753708,
         1.37210411, 45.35759147],
       [ 0.84264218,  0.83138039,  0.83562795, ...,  1.19977144,
         1.3457    , 46.47390772],
       [ 0.84444666,  0.82868655,  0.83209898, ...,  1.16906148,
         1.38365439, 49.95236415]])

In [43]:
df_try.to_numpy().shape

(20, 162)

In [44]:
ftest['test2'].shape

(20, 162)

In [55]:
new_abs = calculate_weighted_averages_by_depth_increment(absorbances, depths, 0.0, 110.0, 5.0)

In [56]:
new_abs

([array([0.84320213, 0.83326291, 0.83190135, 0.83374413, 0.84033444,
         0.82513508, 0.8256238 , 0.82200251, 0.80494589, 0.81890882,
         0.80764813, 0.80988704, 0.79645961, 0.80744545, 0.80530395,
         0.80962274, 0.80769335, 0.81447327, 0.78198947, 0.77952846,
         0.7845375 , 0.80061065, 0.80345236, 0.78320955, 0.81343003,
         0.80906162, 0.8005249 , 0.80101217, 0.80219665, 0.8011089 ,
         0.78595171, 0.80246053, 0.79359893, 0.82514444, 0.80494815,
         0.78960483, 0.79740302, 0.80544252, 0.84440437, 0.81021006,
         0.80622551, 0.82209331, 0.82652482, 0.82851003, 0.82284175,
         0.84857163, 0.79737334, 0.81739247, 0.79963904, 0.83741798,
         0.83060192, 0.81620528, 0.82929685, 0.86479068, 0.81573383,
         0.85281469, 0.81446758, 0.81079668, 0.83334264, 0.84298748,
         0.82740372, 0.84211976, 0.83360694, 0.81908199, 0.84052653,
         0.82655844, 0.81408576, 0.8645727 , 0.84107464, 0.8488161 ,
         0.87637522, 0.87010892, 0

In [57]:
new_for = calculate_weighted_averages_by_depth_increment(forces, depths, 0.0, 110.0, 5.0)

In [58]:
new_for

([43.622264801247084,
  54.22844894616519,
  56.26945619825723,
  47.35321763839417,
  57.35032259845592,
  42.42758695418183,
  48.83684749772854,
  62.90525595910342,
  51.92839400860673,
  56.56425996709616,
  58.2782919133331,
  61.852512000041806,
  63.35867972412724,
  45.774833831991764,
  54.449858553725015,
  61.62177514815387,
  53.806772108679844,
  45.357591465571936,
  46.47390771820532,
  49.952364153027936,
  0.0,
  0.0],
 array([  0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,  50.,
         55.,  60.,  65.,  70.,  75.,  80.,  85.,  90.,  95., 100., 105.]),
 array([  2.5,   7.5,  12.5,  17.5,  22.5,  27.5,  32.5,  37.5,  42.5,
         47.5,  52.5,  57.5,  62.5,  67.5,  72.5,  77.5,  82.5,  87.5,
         92.5,  97.5, 102.5, 107.5]))

In [50]:
d1 = np.array([0, 1, 2])

In [51]:
d1.shape

(3,)

In [52]:
d1.isarray

AttributeError: 'numpy.ndarray' object has no attribute 'isarray'

In [53]:
isinstance(d1, np.ndarray)

True

In [59]:
f.close()

In [60]:
f2 = h5py.File("simulation_test3.h5", "r")

In [61]:
f2.keys()

<KeysViewHDF5 ['session001', 'session002', 'session003', 'session004', 'session005', 'session006', 'session007', 'session008', 'session009', 'session010', 'session011', 'session012', 'session013', 'session014', 'session015', 'session016']>

In [65]:
a_df = f2['session016/cal001/ins001/derived/absorbance_depth'][:]

In [66]:
a_df

array([[ 0.83935804,  0.83305676,  0.8376685 , ...,  1.36088165,
        48.63317944,  0.15863166],
       [ 0.84387714,  0.83036313,  0.83693952, ...,  1.39035685,
        35.05679274,  1.72257577],
       [ 0.84602422,  0.83343758,  0.83982594, ...,  1.3858053 ,
        55.7870185 ,  3.47690163],
       ...,
       [ 0.83826554,  0.8296857 ,  0.83417795, ...,  1.32904363,
        28.64837663, 13.83808339],
       [ 0.83861907,  0.8280884 ,  0.83914984, ...,  1.33324382,
        41.97766192, 15.86856014],
       [ 0.84452369,  0.82860494,  0.8360476 , ...,  1.36184479,
        60.581145  , 17.96647975]])

In [67]:
a_df.shape

(11, 163)

In [68]:
a_df[0]

array([ 0.83935804,  0.83305676,  0.8376685 ,  0.83281214,  0.8396489 ,
        0.82218331,  0.82451123,  0.82224889,  0.80773762,  0.81922528,
        0.80388131,  0.81010513,  0.79583588,  0.80871557,  0.80456534,
        0.80769437,  0.80873287,  0.8112514 ,  0.78077962,  0.78220643,
        0.78223844,  0.80428653,  0.7999432 ,  0.78438344,  0.80830115,
        0.8072022 ,  0.79783343,  0.80370715,  0.8021634 ,  0.8078744 ,
        0.78404456,  0.80417494,  0.79667722,  0.83120281,  0.79974564,
        0.79379344,  0.79843709,  0.80379969,  0.84642425,  0.80741675,
        0.80718313,  0.82654789,  0.82749432,  0.83582091,  0.82583823,
        0.83710122,  0.8019409 ,  0.81515389,  0.80232066,  0.83508784,
        0.82730347,  0.81384313,  0.81890339,  0.86715801,  0.82466501,
        0.848866  ,  0.805117  ,  0.81410253,  0.82584743,  0.84725187,
        0.82819053,  0.85474331,  0.8285835 ,  0.82395093,  0.84286693,
        0.82184532,  0.80800301,  0.86375212,  0.83969238,  0.85

In [69]:
a_abs = a_df[:, :-2]
a_for = a_df[:, -2]
a_dep = a_df[:, -1]

In [70]:
a_abs.shape

(11, 161)

In [71]:
a_for.shape

(11,)

In [72]:
a_dep.shape

(11,)

In [77]:
a_abs_w, bin_mins, bin_centers = calculate_weighted_averages_by_depth_increment(a_abs, a_dep, 0.0, 100.0, 5.0)

In [78]:
a_for_w, fbin_mins, fbin_centers = calculate_weighted_averages_by_depth_increment(a_for, a_dep, 0.0, 100.0, 5.0)

In [79]:
a_abs_w.shape

(20, 161)

In [80]:
a_for_w.shape

(20,)

In [81]:
a_abs_w[-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [82]:
a_df2 = pd.DataFrame(a_df)

In [83]:
a_df2

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,162
0,0.839358,0.833057,0.837669,0.832812,0.839649,0.822183,0.824511,0.822249,0.807738,0.819225,...,1.331201,1.329628,1.247163,1.246966,1.324731,1.222082,1.170343,1.360882,48.633179,0.158632
1,0.843877,0.830363,0.836940,0.838263,0.836624,0.828047,0.819971,0.827272,0.806337,0.813878,...,1.288901,1.312557,1.231270,1.257694,1.373089,1.240980,1.172164,1.390357,35.056793,1.722576
2,0.846024,0.833438,0.839826,0.835821,0.834642,0.828040,0.826609,0.825922,0.802745,0.817361,...,1.304242,1.323879,1.206524,1.271995,1.364121,1.207824,1.182131,1.385805,55.787019,3.476902
3,0.838358,0.830769,0.838128,0.833092,0.837276,0.823370,0.825195,0.820703,0.798907,0.816573,...,1.290840,1.319729,1.236334,1.262251,1.353301,1.237387,1.201435,1.385487,45.736725,5.334732
4,0.840698,0.834978,0.831537,0.837832,0.835951,0.828607,0.822442,0.823617,0.800625,0.814681,...,1.306512,1.334457,1.228128,1.281135,1.343361,1.251955,1.204314,1.374212,33.699003,7.088811
5,0.839393,0.827846,0.834934,0.833633,0.840702,0.820020,0.826629,0.824818,0.800637,0.820017,...,1.281321,1.327745,1.228871,1.235363,1.343662,1.248598,1.188874,1.365633,60.909644,8.833831
6,0.841980,0.828585,0.838505,0.832094,0.835397,0.823769,0.824442,0.826491,0.806060,0.821133,...,1.283831,1.291655,1.218323,1.231118,1.320839,1.210520,1.168790,1.374718,41.683828,10.729176
7,0.844661,0.831340,0.835535,0.831190,0.837283,0.823007,0.822117,0.823697,0.803648,0.815108,...,1.351427,1.314907,1.233840,1.278184,1.326429,1.253645,1.180581,1.349489,27.420750,11.973576
8,0.838266,0.829686,0.834178,0.839331,0.838908,0.826830,0.819684,0.824233,0.800064,0.817637,...,1.331514,1.326723,1.248279,1.243619,1.320512,1.218463,1.186735,1.329044,28.648377,13.838083
9,0.838619,0.828088,0.839150,0.839601,0.837394,0.822449,0.823995,0.820899,0.802505,0.816736,...,1.299128,1.307202,1.222013,1.266246,1.344709,1.208151,1.206200,1.333244,41.977662,15.868560


In [85]:
a_df2.iloc[:,:].to_numpy()

array([[ 0.83935804,  0.83305676,  0.8376685 , ...,  1.36088165,
        48.63317944,  0.15863166],
       [ 0.84387714,  0.83036313,  0.83693952, ...,  1.39035685,
        35.05679274,  1.72257577],
       [ 0.84602422,  0.83343758,  0.83982594, ...,  1.3858053 ,
        55.7870185 ,  3.47690163],
       ...,
       [ 0.83826554,  0.8296857 ,  0.83417795, ...,  1.32904363,
        28.64837663, 13.83808339],
       [ 0.83861907,  0.8280884 ,  0.83914984, ...,  1.33324382,
        41.97766192, 15.86856014],
       [ 0.84452369,  0.82860494,  0.8360476 , ...,  1.36184479,
        60.581145  , 17.96647975]])

In [89]:
a_df2.index.name

In [90]:
a_df2.index

RangeIndex(start=0, stop=11, step=1)

In [92]:
a_df2.index.name = "hi"

In [93]:
a_df2

,0,1,2,3,4,5,6,7,8,9,...,153,154,155,156,157,158,159,160,161,162
hi,,,,,,,,,,,,,,,,,,,,,
0,0.839358,0.833057,0.837669,0.832812,0.839649,0.822183,0.824511,0.822249,0.807738,0.819225,...,1.331201,1.329628,1.247163,1.246966,1.324731,1.222082,1.170343,1.360882,48.633179,0.158632
1,0.843877,0.830363,0.836940,0.838263,0.836624,0.828047,0.819971,0.827272,0.806337,0.813878,...,1.288901,1.312557,1.231270,1.257694,1.373089,1.240980,1.172164,1.390357,35.056793,1.722576
2,0.846024,0.833438,0.839826,0.835821,0.834642,0.828040,0.826609,0.825922,0.802745,0.817361,...,1.304242,1.323879,1.206524,1.271995,1.364121,1.207824,1.182131,1.385805,55.787019,3.476902
3,0.838358,0.830769,0.838128,0.833092,0.837276,0.823370,0.825195,0.820703,0.798907,0.816573,...,1.290840,1.319729,1.236334,1.262251,1.353301,1.237387,1.201435,1.385487,45.736725,5.334732
4,0.840698,0.834978,0.831537,0.837832,0.835951,0.828607,0.822442,0.823617,0.800625,0.814681,...,1.306512,1.334457,1.228128,1.281135,1.343361,1.251955,1.204314,1.374212,33.699003,7.088811
5,0.839393,0.827846,0.834934,0.833633,0.840702,0.820020,0.826629,0.824818,0.800637,0.820017,...,1.281321,1.327745,1.228871,1.235363,1.343662,1.248598,1.188874,1.365633,60.909644,8.833831
6,0.841980,0.828585,0.838505,0.832094,0.835397,0.823769,0.824442,0.826491,0.806060,0.821133,...,1.283831,1.291655,1.218323,1.231118,1.320839,1.210520,1.168790,1.374718,41.683828,10.729176
7,0.844661,0.831340,0.835535,0.831190,0.837283,0.823007,0.822117,0.823697,0.803648,0.815108,...,1.351427,1.314907,1.233840,1.278184,1.326429,1.253645,1.180581,1.349489,27.420750,11.973576
8,0.838266,0.829686,0.834178,0.839331,0.838908,0.826830,0.819684,0.824233,0.800064,0.817637,...,1.331514,1.326723,1.248279,1.243619,1.320512,1.218463,1.186735,1.329044,28.648377,13.838083


In [94]:
a = np.array([1, 2, 3, 4, 5])
b = a[:3]

In [95]:
b

array([1, 2, 3])

In [96]:
a = a[:3]

In [97]:
a

array([1, 2, 3])

In [100]:
wforce, bin_mins3, bin_centers3 = calculate_weighted_averages_by_depth_increment(forces, depths, 0.0, 110.0, 5.0)

In [101]:
wforce.shape

(20,)

In [102]:
bin_mins3.shape

(20,)

In [103]:
bin_centers3.shape

(20,)